In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
        .appName("group04model") \
        .getOrCreate()

In [2]:
import pandas as pd

In [5]:
schemadf = pd.read_csv('/project/ds5559/fa21-group04/data/WiDS_Datathon_2020_Dictionary.csv')

In [3]:
from pyspark.sql.types import *
data_types = {
    'integer': IntegerType(),
    'binary': IntegerType(),
    'numeric': FloatType(),
    'string': StringType()
}

In [6]:
schema = StructType(
    [
        StructField(row['Variable Name'] , 
                    data_types[row['Data Type']], 
                    True) for index, row in schemadf.iterrows()
    ]
)

In [7]:
df = spark.read.format('csv') \
    .schema(schema) \
    .option('header', True) \
    .load('/project/ds5559/fa21-group04/data/training_v2.csv')

In [9]:
import pyspark.sql.functions as F

df = spark.read.option("header",True).csv('/project/ds5559/fa21-group04/data/training_v2.csv')

def drop_null_columns(df):
    null_counts = df.select([((F.count(F.when(F.isnan(c) | F.col(c).isNull() | F.col(c).contains('NA'), c)))*100/df.count()).alias(c)  for c in df.columns]).collect()[0].asDict()
    to_drop = [k for k, v in null_counts.items() if v > 50]
    df = df.drop(*to_drop)
    return df
newdf = drop_null_columns(df)
len(newdf.columns)


112

In [10]:
cleaned_df = newdf.drop('encounter_id').drop('patient_id')